mounting drive to access eval results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

installing necessary modules and trying to clone git library to access nltk 3.10 because collab doesn't have it.. unsuccessful..

In [ ]:
pip install datasets

In [ ]:
pip install rouge

In [ ]:
nltk.download('wordnet')

In [ ]:
!git clone https://github.com/nltk/nltk.git

In [ ]:
%cd nltk
!pip install -r requirements-ci.txt

In [ ]:
!python setup.py install

In [ ]:
!pip install virtualenv

In [ ]:
!virtualenv myenv

In [44]:
!source myenv/bin/activate

In [ ]:
!cd nltk
!pip install .

main code block, calculting f1, meteor, and rouge scores

In [2]:
import json
from datasets import load_metric
import nltk
from nltk.translate import meteor_score
from rouge import Rouge
import nltk.translate.meteor_score as ms


# load all lines into a list
with open('/content/drive/MyDrive/eval_predictions.jsonl', 'r') as file:
    lines = file.readlines()

# list to store the results
success_metrics = []

# starting vals for f1/meteor
total_count = 0
overall_f1 = 0
overall_meteor = 0
overall_rouge = 0
f1_0 = 0
f1_100 = 0

squad_metric = load_metric('squad')
rouge = Rouge()

# process each index in the sample
for index in range(len(lines)):
    # parse the line as a JSON object
    data = json.loads(lines[index])

    # define prediction and reference
    prediction = {"id": data["id"], "prediction_text": data["predicted_answer"]}
    reference = {"id": data["id"], "answers": data["answers"]}

    # F1

    # calc score for this example
    f1_score = squad_metric.compute(predictions=[prediction], references=[reference])
    # update score for whole dataset (running avg)
    overall_f1 = (overall_f1 * total_count + f1_score["f1"]) / (total_count + 1)

    # METEOR

    # redefine pre/ref structure
    tokenized_prediction = prediction["prediction_text"].split()
    tokenized_reference = reference["answers"]["text"][0].split()
    # calc score for this example w/ intermediate vals
    meteor_score_val = meteor_score.meteor_score([tokenized_reference], tokenized_prediction)
    '''meteor_score_val, breakdown = ms.meteor_breakdown([tokenized_reference], tokenized_prediction)
    precision = breakdown.precision
    recall = breakdown.recall
    penalty = breakdown.penalty'''

    # update score for whole dataset (running avg)
    overall_meteor = (overall_meteor * total_count + meteor_score_val) / (total_count + 1)

    # ROUGE (BIGRAM)

    # redefine pred/ref structure
    prediction_sentences = prediction["prediction_text"]
    reference_sentences = " ".join(reference["answers"]["text"])
    # calc score for this example
    rouge_score = rouge.get_scores(prediction_sentences, reference_sentences)[0]["rouge-2"]["f"]
    # update score for whole dataset (running avg)
    overall_rouge = (overall_rouge * total_count + rouge_score) / (total_count + 1)

    # ANALYSIS

    # for extreme f1 values, are the meteor/rouge scores capturing better results?
    if f1_score["f1"] == 0.0:
      f1_0 += 1
      '''print(f"ref:", tokenized_reference, reference)
      print(f"pred:", tokenized_prediction, prediction)
      print(meteor_score_val)
      print(rouge_score)'''
    if f1_score["f1"] == 100.0:
      f1_100 += 1
      '''if meteor_score_val != 1:
        print(f"ref:", tokenized_reference, reference)
        print(f"pred:", tokenized_prediction, prediction)
        print(meteor_score_val)
        print(precision, recall, penalty)'''
    success_metrics.append({"data": data, "f1": f1_score["f1"], "meteor": meteor_score_val, "rouge": rouge_score})

    # increment count
    total_count += 1

    '''if total_count > 10:
      break'''

# RESULTS

# write to the output file
with open('alt_f1_metrics.jsonl', 'w') as outfile:
    for example in success_metrics:
        outfile.write(json.dumps(example, ensure_ascii=False))
        outfile.write('\n')

# print summary
print("Overall F1 Score:", overall_f1)
print("Overall METEOR Score:", overall_meteor)
print("Overall ROUGE Score (Bigrams):", overall_rouge)
print(f"Number of f1 failures:", f1_0)
print(f"Number of f1 successes:", f1_100)

Overall F1 Score: 86.41892946974727
Overall METEOR Score: 0.6185034925611023
Overall ROUGE Score (Bigrams): 0.3614723669186699
Number of f1 failures: 850
Number of f1 successes: 8323


convert json to xlsx

In [8]:
!pip install pandas openpyxl

In [13]:
import pandas as pd
import json

# empty list to store JSON objects
data_list = []

# load each line of the JSON file separately
with open('/content/alt_f1_metrics.jsonl') as f:
    for line in f:
        data_list.append(json.loads(line))

# convert to pandas df
df = pd.DataFrame(data_list)

# split up the first data columnn into its keys
df_normalized = pd.json_normalize(df['data'])
# concatenate original df with the new split df
df_result = pd.concat([df, df_normalized], axis=1)
# drop the original combined data column
df_result = df_result.drop('data', axis=1)

# convert to excel
df_result.to_excel('alt_f1_metrics.xlsx', index=False, engine='openpyxl')